In [120]:
import json
from elasticsearch import Elasticsearch
from elasticsearch.helpers import parallel_bulk
import os
from tqdm import tqdm_notebook as tqdm
import time
from lxml import etree
from sklearn.metrics import r2_score
from datetime import timedelta
import numpy as np

In [121]:
def create_es_action(index, doc_id, document):
    return {
        '_index': index,
        '_id': doc_id,
        '_source': document
    }

def pretty_print_result(search_result, fields=None):
    if fields is None:
        fields = []
    res = search_result['hits']
    print(f'Total documents: {res["total"]["value"]}')
    for hit in res['hits']:
        print(f'Doc {hit["_id"]}, score is {hit["_score"]}')
        for field in fields:
            print(f'{field}: {hit["_source"][field]}')


def get_score(search_result):
    res = {}
    for hit in search_result['hits']['hits']:
        res[hit["_id"]] = hit["_score"]
    return res


class Index:
    def __init__(self, index, settings):
        self.index_name = index
        self.settings = settings
        self.es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'timeout': 360}])
        if not self.es.indices.exists(index=index):
            self.es.indices.create(index=index, body=settings)

    def es_actions_generator(self, path_to_docs):
        for doc_name in tqdm(os.listdir(path_to_docs)):
            with open(f"res/json/{doc_name}", "r", encoding="utf-8") as inf:
                doc_id = int(''.join(list(filter(str.isdigit, doc_name))))
                doc = json.load(inf)
            yield create_es_action(self.index_name, doc_id, doc)

    def add_documents(self, path_to_docs):
        for ok, result in parallel_bulk(self.es, self.es_actions_generator(path_to_docs), queue_size=4, thread_count=4,
                                        chunk_size=1000):
            if not ok:
                print(result)

    def get_doc_by_id(self, doc_id):
        return self.es.get(index=self.index_name, id=doc_id)['_source']

    def search(self, query, *args):
        return self.es.search(index=self.index_name, body=query, size=1000)
        # note that size set to 20 just because default value is 10 and we know that we have 12 docs and 10 < 12 < 20


In [122]:
settings_1 = {
    "mappings": {
        "properties": {
            "text": {
                "type": "text"
            }
        }
    }
}

In [123]:
index = Index("docs", settings_1)

In [109]:
start = time.time()
index.add_documents("res/json")
elapsed = time.time() - start
print(str(timedelta(seconds=elapsed)))


0:05:27.750718


In [124]:
class Query:
    def __init__(self, task_id, query, relevant_docs):
        self.task_id = task_id
        self.query = query
        self.relevant_docs = relevant_docs

    def json_query(self):
        return {
    'query': {
        'bool': {
            'should': [
                {
                    'match': {
                        'text': self.query
                    }
                }
            ]
        }
    }
}

In [131]:
class SearchQualityChecker:
    def __init__(self, queries, index):
        self.queries = queries
        self.index = index
        self.results = {}
        
    def save_results(self):
        for q in queries:
            res = index.search(q.json_query())
            print(f"\n{q.query}")
            print(self.r_precision(q, get_score(res)))
    
    def r_precision(self, query, search_res_score):
        r = 0
        for doc in query.relevant_docs:
            if doc in search_res_score:
                r += 1
        R = len(query.relevant_docs)
        return r / R if R != 0 else 0 if len(search_res_score) > 0 else 1
    
    def 

In [132]:
def get_relevance():
    res = {}
    xml_tree = etree.parse("data/or_relevant-minus_table.xml")
    root = xml_tree.getroot()
    for task in root.getchildren():
        relevant_docs = set()
        for document in task.getchildren():
            if document.get("relevance") == "vital":
                relevant_docs.add(document.get("id"))
        res[task.get("id")] = relevant_docs
    print(len(res))
    return res


def generate_queries_plain_texts():
    relevances = get_relevance()
    xml_tree = etree.parse("data/web2008_adhoc.xml")
    root = xml_tree.getroot()
    res = []
    for task in root.getchildren():
        if task.get("id") is not None:
            for query_text in task.getchildren():
                try:
                    res.append(Query(task.get("id"), query_text.text, relevances[task.get("id")]))
                except:
                    pass
    print(len(res))
    return res

In [133]:
queries = generate_queries_plain_texts()
quality_checker = SearchQualityChecker(queries, index)
quality_checker.save_results()

547
547

корозия металла
0.4807692307692308

авиашоп
0.0

византийские источники о руси
0.8461538461538461

Ю. Савичева - Прости за любовь текст
0.5555555555555556

единый государственный реестр
0.6428571428571429

дородовый бандаж
0.5588235294117647

планета суши
0.8409090909090909

Battlestar Galactica
0.9411764705882353

инвесткапиталбанк
0.0

Конфактура Дюпютрена
0.0

Финансовый директор новой эпохи
0.48148148148148145

биография все фабрикантов седьмой фабрики звёзд
0.8

club turtas
1.0

нормальные размеры таза
0.4

бюджет управляющей компании ЖКХ
0.8333333333333334

выборский клуб любителей собак
0

борьба с терроризмом
0.8529411764705882

скачать бесплатно аудиокнигу
0.3333333333333333

Функции обработки массивов
0.6875

тюнинг ваз 2110
0.896551724137931

Готовый БИЗНЕС ПЛАН
0.76

Юлай Салаватович
0

велькин владимир кафедра атомной энергетики угту упи
0.7391304347826086

детский шатер
0.7948717948717948

показатель экономический
0.45544554455445546

элюминирование, уфа
0

росно


расхода газовоздушной смеси автомобиля
0.8666666666666667

Ремолан
0.0

удельный вес ПГС
1.0

стадии и факторы свертывания крови
0.75

поздравления с двадцатилетием
0.9142857142857143

скачать книгу От игр к играм. Математическое введение
0

альт инвест
0.8421052631578947

hi-fi
0.9203539823008849

карта турции
0.5769230769230769

контактор КМ3-20
0.43243243243243246

поздравления с днем рождения
0.9078947368421053

как сделать обычную прогррамму портативной
0

jimm для нокиа N73
0.75

кто архитектор Кремля?
0.2

Britney_Spears туц скачать
0.0

греческий архитектурный стиль
0.5714285714285714

познание как деятельность.
0.631578947368421

французский кутюрье, у которого одевалась кеннеди
1.0

Сбербанк
0.5

Интересные тесты
0.5714285714285714

Ключевые проблемы и объяснительные принципы психологии.
0.7755102040816326

педагогический менеджмент
0.4634146341463415

лодочный мотор ямаха
0.36363636363636365

Нива-шевролет
0.75

аэропорт кольцово
0.875

сонник
0.855072463768116

вакансии от


Рак
0.65

автоматические настройки интернета для Билайна
0.8

иероглиф любовь
0.75

стихотворения Пушкин памятник
0.84

чипсеты от NVIDIA
0.8072289156626506

Как правильно оформить платежное поручение по налогу на прибыль с дивидендов у Российской организации
0.8571428571428571

туристическая компания Пегас
0.8571428571428571

капитал тур
0.7346938775510204

вжик аватар
0.9285714285714286

пежо 308 универсуал фото
0.8656716417910447

выставка в Московском Кремле
0.25

основные понятия в педагогике
0.6428571428571429

фильтры Аквафор Екатеринбург
0

строительные организации
0.6714285714285714

снижение иммунитета у ребенка
0.6756756756756757

работа
0.44954128440366975

мебель vox
0.7619047619047619

обои racsh
0.7777777777777778

Значение причёски для создания имиджего образа
0.42857142857142855

укрепление зубов
0.8

MTA AS267N Чиллер холодопроивод. 267 кВт, фреон R407C
1.0

обоснование второго начала термодинамики
0.5833333333333334

сообщение об открытии счета скачать
0.81818181818

In [160]:
from pymystem3 import Mystem
from nltk.corpus import stopwords
import nltk
#nltk.download('punkt')
#nltk.download('stopwords')
from string import punctuation
from nltk.corpus import stopwords

russian_stopwords = stopwords.words("russian")
english_stopwords = stopwords.words("english")
black_list = ["°", "№", "©", "...", "//", "://", "</", "\">", "=\"", "=\'", "\r", "\n", "\t"]
stem = Mystem()

def lemmatize(text):
    words = nltk.word_tokenize(text.lower())
    tokens = []
    for word in words:
        tokens.extend(stem.lemmatize(word))
    tokens = [token for token in tokens if token != " " and token.strip() not in punctuation \
                  and token not in russian_stopwords and token not in english_stopwords \
                  and token not in black_list \
                  and token.find("\r") == -1 \
                  and token.find("\n") == -1 \
                  and token.find("\t") == -1 \
                  and not (token.isdigit() and len(token) == 1)]
    return ' '.join(tokens)

def generate_queries_lemmas():
    relevances = get_relevance()
    xml_tree = etree.parse("data/web2008_adhoc.xml")
    root = xml_tree.getroot()
    res = []
    for task in tqdm(root.getchildren()):
        if task.get("id") is not None:
            for query_text in task.getchildren():
                try:
                    res.append(Query(task.get("id"), lemmatize(query_text.text), relevances[task.get("id")]))
                except:
                    pass
    print(len(res))
    return res

In [ ]:
queries_lemmas = generate_queries_lemmas()

547


In [156]:
queries_lemmas[0].query

'корозия металла'

In [157]:
lemmatize('корозия металла')

'корозия металл'